<a href="https://colab.research.google.com/github/pejmanrasti/MiFoBio2021/blob/main/2_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Make your fist experience with Tensorflow-Keras**
Our goal is to construct and train an artificial neural network on thousands of images of handwritten digits so that it may successfully identify others when presented. The data that will be incorporated is the MNIST database which contains 60,000 images for training and 10,000 test images.

## Loading Training and Validation Data

The MNIST dataset is conveniently bundled within Keras, and we can easily analyze some of its features in Python.

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import os
import cv2
import random
import numpy as np
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential # Model type to be used
from tensorflow.keras.layers import Dense, Activation, Dropout# Make Fully connected (FC) layers
from tensorflow.keras.utils import to_categorical # NumPy related tools
from tensorflow.keras.callbacks import TensorBoard  #Visulization of Accuracy and loss


import numpy as np                   # advanced math library
import matplotlib.pyplot as plt      # MATLAB like plotting routines
import random                        # for generating random numbers


In [ ]:
print(tf.__version__)

# **Make a connection between colab and your google drive Where your data are saved.**

In [ ]:
from google.colab import drive
root = '/content/gdrive/'
drive.mount( root )

In [ ]:
# create permanent directory in gdrive
images = r'/My Drive/DIRECTORY OF YOUR OWN DATA/'
os.makedirs(root+images, exist_ok=True)
os.listdir(root+images)

# **Reading and saving data** 
What we need is a training data directory (and/or validation data directory)  containing one subdirectory per image class, filled with images. For example: 

```
Animals/
    train/
        dogs/
            dog001.jpg
            dog002.jpg
            ...
        cats/
            cat001.jpg
            cat002.jpg
            ...
```



In [ ]:
DATADIR = root+images
CATEGORIES = os.listdir(DATADIR)
print(CATEGORIES)

In [ ]:
training_data = []
IMG_SIZE_H=224 # you need to set up a numerical value here. Useful to resize to normalize data size
IMG_SIZE_W=224 # you need to set up a numerical value here. Useful to resize to normalize data size
def create_training_data():
    for category in CATEGORIES:

        path = os.path.join(DATADIR,category)  # create path to the labels
        class_num = CATEGORIES.index(category)  # giving an index to each class (subfolder) -- (0 to the first subfolder, 1 to the first subfolder, etc). 

        for img in os.listdir(path):  # iterate over each image per psubfolder
          if img.endswith('.tif'):
            img_array = cv2.imread(os.path.join(path,img),cv2.IMREAD_GRAYSCALE)  # convert to array 
            new_array = cv2.resize(img_array, (IMG_SIZE_H, IMG_SIZE_W))  # resize to normalize data size
            training_data.append([new_array, class_num])  # add this to our training_data


In [ ]:
create_training_data()  # Calling the function for reading images and labels
print(len(training_data)) # Printing the size of the database

Preparation of data for importing to Keras

In [ ]:
random.shuffle(training_data)
X = []  # An Array for images
y = []  # An Array for labels

for features,label in training_data:   # Seperation of iamegs and labels
    X.append(features)
    y.append(label)
print(np.array(X).shape) # Print the size of the database

In [ ]:
X = np.array(X).reshape(-1, IMG_SIZE_H, IMG_SIZE_W, 1)  # Reshape data in a form that is suitable for keras
print(X.shape) # Print the size of the database

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

In [ ]:
# Display 3 images 
plt.subplot(131)
plt.imshow(X[0,:,:,:])
plt.axis("off")
plt.subplot(132)
plt.imshow(X[10,:,:,:])
plt.axis("off")
plt.subplot(133)
plt.imshow(X[30,:,:,:]) 
plt.axis("off")
# show the plot
plt.show()

# **Importing necessary Libraries**

# **Let's go furter with introducing CNN**
Before, we built a network that accepts the normalized pixel values of each value and operates soley on those values. What if we could instead feed different features (e.g. curvature, edges) of each image into a network, and have the network learn which features are important for classifying an image?

This possible through convolution! Convolution applies kernels (filters) that traverse through each image and generate feature maps.

In [ ]:
# import some additional tools

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten

In [ ]:
modelCNN = Sequential([
    
    # Convolution Layer 1
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 1)), # 32 different 3x3 kernels -- so 32 feature maps
    MaxPooling2D(pool_size=(2, 2)), # Pool the max values over a 2x2 kernel

    # Convolution Layer 2
    Conv2D(64, (3, 3), activation='relu'), # 64 different 3x3 kernels 
    MaxPooling2D(pool_size=(2, 2)),

    # Convolution Layer 3
    Conv2D(128, (3, 3), activation='relu'), # 128 different 3x3 kernels

    Flatten(), # Flatten final 7x7x128 output matrix into a 1024-length vector 

    # Fully Connected Layer 4
    Dense(512), # 512 FCN nodes
    Activation('relu'),
    Dropout(0.2),
    Dense(10),
    Activation('softmax'),
])
modelCNN.summary()

In [ ]:
import datetime
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
modelCNN.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam',
              metrics=['accuracy'])

modelCNN.fit(X_train, y_train, 
          validation_data=(X_test, y_test),
          epochs=20, batch_size=10,
          verbose=1,
          callbacks=[tensorboard_callback])

**Evaluation and Prediction**

We can use our model to make a prediction on new images.

In [ ]:
modelCNN.evaluate(X_test,Y_test,verbose=0) #Evaluation of the model on the test dataset

In [ ]:
modelCNN.predict_classes(X_test,verbose=0) # Prediction of classes

In [ ]:
# Prediction of classes of a single image
img=X_test[1,:,:,:]
img = np.array(img).reshape(-1, 224, 224, 1)
output = modelCNN.predict_classes(img)
print('The predicted label is: ',output[0])
print('The real label is: ',y_test[1])